In [1]:
from dataset import Dataset
from classificadores import PerceptronSimples, MultiLayerPerceptron

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from typing import Union

In [2]:
# Conjunto de dados Câncer de Mama =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\wdbc.data", 
    label_column = 1,
    delimiter = ",",  
    column_names = ["ID", "Diagnosis", "radius1", "texture1", "perimeter1", "area1", "smoothness1", "compactness1", "concavity1", "concave_points1", "symmetry1", "fractal_dimension1", "radius2", "texture2", "perimeter2", "area2", "smoothness2", "compactness2", "concavity2", "concave_points2", "symmetry2", "fractal_dimension2", "radius3", "texture3", "perimeter3", "area3", "smoothness3", "compactness3", "concavity3", "concave_points3", "symmetry3", "fractal_dimension3"]
).ensure_numeric_labels().remove_features(["ID"]).normalize()

print(dataset)
dataset._label_index_to_name

Dataset(instâncias=569, features=30, classes=2)


{0: 'M', 1: 'B'}

In [3]:
# Conjunto de dados Coluna Vertebral =================================================
dataset = Dataset.from_file( 
    filepath = r"datasets\column_3C.dat", 
    label_column = -1,
    delimiter = " ",  
    column_names = ["pelvic incidence", "pelvic tilt", "lumbar lordosis angle", "sacral slope", "pelvic radius", "degree spondylolisthesis", "class"]
).ensure_numeric_labels().normalize()

print(dataset)
dataset._label_index_to_name

Dataset(instâncias=310, features=6, classes=3)


{0: 'DH', 1: 'SL', 2: 'NO'}

In [4]:
dataset.data

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,degree spondylolisthesis,class
0,-0.288580,0.039657,-0.541614,-0.498242,-0.385095,-0.949674,0.0
1,-0.750965,-0.406574,-0.802756,-0.710716,-0.046564,-0.927281,0.0
2,-0.176698,0.027867,-0.354036,-0.384786,-0.227659,-0.964944,0.0
3,-0.167631,0.114684,-0.457491,-0.421247,-0.316271,-0.896322,0.0
4,-0.545525,-0.421222,-0.743691,-0.506015,-0.180772,-0.911639,0.0
...,...,...,...,...,...,...,...
305,-0.580440,-0.279385,-0.606229,-0.612808,0.018819,-0.968296,2.0
306,-0.463927,-0.025723,-0.727582,-0.632612,-0.047424,-0.950466,2.0
307,-0.319059,0.044659,-0.424199,-0.530261,0.195612,-0.961127,2.0
308,-0.631559,-0.455520,-0.506354,-0.570794,0.042478,-0.947533,2.0


In [5]:
dataset.determination_matrix()

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,degree spondylolisthesis,class
pelvic incidence,1.000000,0.395875,0.514499,0.664158,0.061248,0.407979,0.000845
pelvic tilt,0.395875,1.000000,0.187282,0.003885,0.001067,0.158277,0.044820
lumbar lordosis angle,0.514499,0.187282,1.000000,0.358069,0.006459,0.284798,0.001351
sacral slope,0.664158,0.003885,0.358069,1.000000,0.117064,0.274127,0.014523
pelvic radius,0.061248,0.001067,0.006459,0.117064,1.000000,0.000680,0.055102
degree spondylolisthesis,0.407979,0.158277,0.284798,0.274127,0.000680,1.000000,0.014286
class,0.000845,0.044820,0.001351,0.014523,0.055102,0.014286,1.000000


In [6]:
dataset.vectorize_labels()
dataset.label_encodings

{np.float64(0.0): array([ 1., -1., -1.]),
 np.float64(1.0): array([-1.,  1., -1.]),
 np.float64(2.0): array([-1., -1.,  1.])}

In [7]:
# Separa o conjunto de dados em treinamento e teste
train_dataset, test_dataset = dataset.split()
train_dataset : Dataset 
test_dataset : Dataset

In [85]:
class ExtremeLearningMachine:
    def __init__( self, train_dataset : Dataset ):
        self.train_dataset = train_dataset

        self.q = 4                                  # Número de neurônios ocultos
        self.p = train_dataset.features_count + 1   # Número de entradas da rede
        self.m = train_dataset.class_count          # Número de classes do dataset
        self.N = len(train_dataset)                 # Número de instâncias do conjunto de treino

        # Função de ativação
        self.phi = lambda u: (1 - np.exp(-u)) / (1 + np.exp(-u))

        # Incializa as matrizes de pesos
        self.W = np.random.normal( size=(self.q, self.p) )     # Pesos da camada oculta (q x p)
        self.M = np.random.normal( size=(self.m, self.q+1) )   # Pesos da camada de saída (m, q+1)

    def train( self ):
        
        Z = np.zeros((self.q+1, self.N))    # Matriz de entradas para a camada de saída de cada amostra (q+1, N)
        D = np.zeros((self.m, self.N))      # Matriz dos vetores de saída desejada para cada amostra (m, N)

        # Preenche as matrizes com os dados de cada instância de treinamento
        for index, *features, classe in self.train_dataset:
            # Vetor que representa a classe 
            real_output = self.train_dataset.encode_label( classe )

            X = np.r_[features, -1]     # Vetor de entrada da camada oculta
            u = self.W @ X              # Ativação da camada oculta
            y = self.phi(u)             # Saída da camada oculta (1xq)

            Z[:, index] = np.r_[y, -1]  # Adiciona a saída da camada oculta na coluna adequada da amostra atual
            D[:, index] = real_output   # Adiciona o vetor de saída desejada na coluna adequada da amostra atual
        
        # Atualiza a matriz de pesos de saída 
        inversa = np.linalg.inv( Z @ Z.T ) # (q+1, q+1)

        self.M = (
            D @ Z.T     # (m, N) x (N, q+1) = (m, q+1)
            @ inversa   # (m, q+1) x (q+1, q+1) = (m, q+1)
        )
    
    def predict( self, features : np.ndarray ) -> Union[float, int]:
        # Monta o vetor de entrada
        x_bias = np.r_[features, -1]

        u = self.W @ x_bias # Ativação de cada neurônio oculto
        y = self.phi( u )   # Saída dos neurônios ocultos

        z = np.r_[ y, -1 ]  # Prepara as entradas para os neurônios de saída
        o = self.M @ z      # Calcula a saída dos neurônios de saída

        # Monta um vetor de predição baseado no argmax da saída da rede
        predicted_output = -np.ones_like(o)
        predicted_output[ np.argmax(o) ] = +1

        # Retorna a classe correspondente ao vetor predito pela rede
        return self.train_dataset.decode_vector( predicted_output ) 

In [ ]:
q = 4
p = train_dataset.features_count + 1
m = train_dataset.class_count
N = len(train_dataset)

activation = lambda u: (1 - np.exp(-u)) / (1 + np.exp(-u))

W = np.random.normal( size=(q, p) )     # Pesos da camada oculta (q x p)
M = np.random.normal( size=(m, q+1) )   # Pesos da camada de saída (m, q+1)

Z = np.zeros((q+1, N))                  # Matriz de entrada para a camada de saída para cada amostra (q+1, N)
D = np.zeros((m, N))                    # Matriz dos vetores de saída desejada para cada amostra (m, N)

# Calcula as matrizes para cada amostra de treinamento
for index, *features, classe in train_dataset:
    # Vetor que representa a classe 
    real_output = train_dataset.encode_label( classe )

    X = np.r_[features, -1]     # Vetor de entrada da camada oculta
    u = W @ X                   # Ativação da camada oculta
    y = activation(u)           # Saída da camada oculta (1xq)

    Z[:, index] = np.r_[y, -1]  # Lembrar de adicionar o -1 do bias
    D[:, index] = real_output

# Atualiza a matriz de pesos da saída 
inversa = np.linalg.inv( Z @ Z.T ) # (q+1, q+1)
M = (
    D @ Z.T         # (m, q+1)
      @ inversa     # (m, q+1)
)


In [80]:
MLP = MultiLayerPerceptron( train_dataset )
MLP.train( 500 )

for index, *point_test, classe in test_dataset:
    classe_prevista = MLP.predict( point_test )
    print(f"{index}] Previu {classe_prevista} e era {classe} [{classe_prevista == classe}]")

Época 0: 111 erros.
Época 25: 45 erros.
Época 50: 39 erros.
Época 75: 42 erros.
Época 100: 39 erros.
Época 125: 37 erros.
Época 150: 38 erros.
Época 175: 35 erros.
Época 200: 31 erros.
Época 225: 34 erros.
Época 250: 39 erros.
Época 275: 36 erros.
Época 300: 37 erros.
Época 325: 33 erros.
Época 350: 32 erros.
Época 375: 29 erros.
Época 400: 32 erros.
Época 425: 35 erros.
Época 450: 32 erros.
Época 475: 33 erros.
Treinamento encerrado com 36 erros após 499 épocas.
0] Previu 1.0 e era 1.0 [True]
1] Previu 0.0 e era 0.0 [True]
2] Previu 1.0 e era 1.0 [True]
3] Previu 2.0 e era 2.0 [True]
4] Previu 0.0 e era 0.0 [True]
5] Previu 0.0 e era 0.0 [True]
6] Previu 1.0 e era 1.0 [True]
7] Previu 1.0 e era 1.0 [True]
8] Previu 1.0 e era 1.0 [True]
9] Previu 1.0 e era 1.0 [True]
10] Previu 1.0 e era 1.0 [True]
11] Previu 1.0 e era 1.0 [True]
12] Previu 1.0 e era 1.0 [True]
13] Previu 0.0 e era 0.0 [True]
14] Previu 0.0 e era 2.0 [False]
15] Previu 1.0 e era 1.0 [True]
16] Previu 2.0 e era 0.0 [Fal

In [9]:
PS = PerceptronSimples( train_dataset )
PS.train( 500 )

for index, *point_test, classe in test_dataset:
    classe_prevista = PS.predict( point_test )
    print(f"{index}] Previu {classe_prevista} e era {classe} [{classe_prevista == classe}]")

Época 0: erros: 161
Época 25: erros: 75
Época 50: erros: 84
Época 75: erros: 85
Época 100: erros: 72
Época 125: erros: 76
Época 150: erros: 71
Época 175: erros: 70
Época 200: erros: 73
Época 225: erros: 82
Época 250: erros: 74
Época 275: erros: 77
Época 300: erros: 68
Época 325: erros: 74
Época 350: erros: 72
Época 375: erros: 72
Época 400: erros: 62
Época 425: erros: 66
Época 450: erros: 76
Época 475: erros: 73
Treinamento encerrado com 70 erros após 499 épocas.
0] Previu 1.0 e era 1.0 [True]
1] Previu 0.0 e era 0.0 [True]
2] Previu 1.0 e era 1.0 [True]
3] Previu 2.0 e era 2.0 [True]
4] Previu 0.0 e era 0.0 [True]
5] Previu 2.0 e era 0.0 [False]
6] Previu 1.0 e era 1.0 [True]
7] Previu 1.0 e era 1.0 [True]
8] Previu 1.0 e era 1.0 [True]
9] Previu 1.0 e era 1.0 [True]
10] Previu 1.0 e era 1.0 [True]
11] Previu 1.0 e era 1.0 [True]
12] Previu 1.0 e era 1.0 [True]
13] Previu 2.0 e era 0.0 [False]
14] Previu 0.0 e era 2.0 [False]
15] Previu 1.0 e era 1.0 [True]
16] Previu 2.0 e era 0.0 [F